In [49]:
import tensorflow as tf
import sklearn as skl
import pandas as pd
from gensim.models.word2vec import Word2Vec
from operator import itemgetter


tag = pd.read_csv('./data/tags_large.csv')
tag_df = pd.DataFrame(tag)

# 태그 데이터 프레임을 만듭니다.

tag_df.head(10)

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195
5,14,318,justice,1442615192
6,14,480,Dinosaurs,1443148563
7,14,593,psychothriller,1444014286
8,14,1682,philosophy,1442615158
9,14,1682,surveillance,1442615167


In [3]:
my_array = []
tmp_array = []
tmp_for_movie_id = 110
tmp_for_user_id = 14

for userId, movieId, tag, time in tag_df.values:

    global lower_tag
    if tmp_for_user_id == userId:
        if tmp_for_movie_id == movieId:
            if not (type(tag) is float):
                lower_tag = tag.lower()
            data_dict = {'time': time, 'tag': lower_tag}
            tmp_array.append(data_dict)

        else:
            sorted_list = sorted(tmp_array, key=itemgetter('time'))
            sorted_tag_list = []
            for i in sorted_list:
                sorted_tag_list.append(i.get('tag'))
            my_array.append(list(sorted_tag_list))
            tmp_for_movie_id = movieId
            tmp_array.clear()
            if not (type(tag) is float):
                lower_tag = tag.lower()
            tmp_array.append({'time': time, 'tag': lower_tag})

    else:
        tmp_for_user_id = userId

for i in range(0, 10):
    print(my_array[i])


['medieval', 'epic']
['sci-fi', 'space action']
['justice', 'imdb top 250']
['dinosaurs']
['psychothriller']
['philosophy', 'surveillance']
['epic']
['pixar']
['dinosaurs']
['sci-fi', 'harrison ford', 'must see']


In [5]:
model = Word2Vec(my_array, 
                 size=256, 
                 window=3, 
                 min_count=50, 
                 workers=4, 
                 iter=100, 
                 sg=1)

model.init_sims(replace=True)

In [6]:
model.wv.most_similar("marvel")

C:\Users\ASUS\Desktop\Github\Movie-Recommender\venv\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('marvel cinematic universe', 0.5729436278343201),
 ('superhero', 0.553860068321228),
 ('mcu', 0.5532954931259155),
 ('the avengers', 0.5270558595657349),
 ('thor', 0.5139960050582886),
 ('comic book', 0.5026129484176636),
 ('spider-man', 0.4775143563747406),
 ('x-men', 0.47613194584846497),
 ('iron man', 0.47003835439682007),
 ('captain america', 0.45416098833084106)]

In [78]:
rating = pd.read_csv('./data/ratings_large.csv')
rating_df = pd.DataFrame(rating)

rating_df = rating_df.drop('userId', axis=1)
rating_df = rating_df.drop('timestamp', axis=1)
rating_df = rating_df.sort_values(by=['movieId'])
rating_df = rating_df.groupby(['movieId'], as_index=False).mean()
df_over_three = rating_df[rating_df['rating'] > 3]
df_over_three = df_over_three.drop('rating', axis=1)
df_over_three.head(20)

,movieId
0,1
1,2
2,3
4,5
5,6
6,7
7,8
8,9
9,10
10,11


In [23]:
movieTag = tag_df[['movieId', 'tag']].sort_values(by=['movieId'])


def get_movie_tag(test_movie_id):
    tag_count = {}
    for movieId, tag in movieTag.values:
        global lower_tag
        if not (type(tag) is float):
            lower_tag = tag.lower()
        if test_movie_id == movieId:

            if not (lower_tag in tag_count):
                tag_count[lower_tag] = 1
            else:
                tempTagCount = tag_count[lower_tag]
                tag_count[lower_tag] = tempTagCount + 1

    sorted_list = sorted(tag_count.items(), key=lambda t: t[1], reverse=True)

    semantic = []
    for tag, count in sorted_list:
        semantic.append(tag)
        if len(semantic) > 2:
            break
    return semantic


In [160]:
over_three_list = []
for i in df_over_three.values:
    over_three_list.append(i[0])

semantic_df = tag_df[['movieId', 'tag']].sort_values(by=['movieId'])
semantic_df = semantic_df[semantic_df['movieId'].isin(over_three_list)]
semantic_df

['Lion King, The (1994)']  영화의 유효 태그는 ['disney', 'animation', 'coming of age']

['Lion King, The (1994)'] ['Adventure|Animation|Children|Drama|Musical|IMAX'] 와 유사한 추천영화 목록입니다.


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
12,13,Balto (1995),Adventure|Animation|Children
26,27,Now and Then (1995),Children|Drama
66,67,Two Bits (1995),Drama
85,86,White Squall (1996),Action|Adventure|Drama
92,94,Beautiful Girls (1996),Comedy|Drama|Romance
105,107,Muppet Treasure Island (1996),Adventure|Children|Comedy|Musical
217,219,"Cure, The (1995)",Drama
243,246,Hoop Dreams (1994),Documentary
244,247,Heavenly Creatures (1994),Crime|Drama


In [ ]:
# -----------------TEST SECTION----------------
movie = pd.read_csv('./data/movies_large.csv')
movie_df = pd.DataFrame(movie)


def recommend(movieId):
    result = []
    recommend_df = semantic_df[semantic_df['tag'].isin(get_movie_tag(movieId))]
    recommend_df = recommend_df.drop('tag', axis=1)
    recommend_df = recommend_df.drop_duplicates()
    for data in recommend_df.values:
        result.append(data[0])
    return result


['Lion King, The (1994)']  영화의 유효 태그는 ['disney', 'animation', 'coming of age']

['Lion King, The (1994)'] ['Adventure|Animation|Children|Drama|Musical|IMAX'] 와 유사한 추천영화 목록입니다.


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
12,13,Balto (1995),Adventure|Animation|Children
26,27,Now and Then (1995),Children|Drama
66,67,Two Bits (1995),Drama
85,86,White Squall (1996),Action|Adventure|Drama
92,94,Beautiful Girls (1996),Comedy|Drama|Romance
105,107,Muppet Treasure Island (1996),Adventure|Children|Comedy|Musical
217,219,"Cure, The (1995)",Drama
243,246,Hoop Dreams (1994),Documentary
244,247,Heavenly Creatures (1994),Crime|Drama


In [ ]:

def id_to_name(movieId):
    return movie_df.loc[movie_df['movieId'] == movieId]


def id_to_names(movieId):
    return movie_df.loc[movie_df['movieId'].isin(movieId)]


user_select_movieId = 2641

user_select_movie_title = id_to_name(user_select_movieId)
print(user_select_movie_title['title'].values, ' 영화의 유효 태그는', get_movie_tag(user_select_movieId))
print('')
print(user_select_movie_title['title'].values,
      user_select_movie_title['genres'].values,
      '와 유사한 추천영화 목록입니다.')

id_to_names(recommend(user_select_movieId))


['Superman IV: The Quest for Peace (1987)']  영화의 유효 태그는 ['superhero', 'dc comics', 'nuclear weapons']

['Superman IV: The Quest for Peace (1987)'] ['Action|Adventure|Sci-Fi'] 와 유사한 추천영화 목록입니다.


,movieId,title,genres
313,316,Stargate (1994),Action|Adventure|Sci-Fi
363,367,"Mask, The (1994)",Action|Comedy|Crime|Fantasy
583,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
586,592,Batman (1989),Action|Crime|Thriller
737,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War
789,802,Phenomenon (1996),Drama|Romance
1184,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
1346,1377,Batman Returns (1992),Action|Crime
1563,1616,"Peacemaker, The (1997)",Action|Thriller|War
2011,2094,"Rocketeer, The (1991)",Action|Adventure|Sci-Fi
